TypeError: loop of ufunc does not support argument 0 of type float which has no callable exp method

In [ ]:
# Random Forest
from collections import Counter

class DecisionTreeFromScratch:
    def __init__(self, max_depth=10, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
    
    def fit(self, X, y):
        self.root = self._build_tree(X, y)
    
    def predict(self, X):
        return [self._traverse_tree(x, self.root) for x in X]
    
    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        # Stopping criteria
        if depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        # Get the best split
        best_feature, best_thresh = self._best_split(X, y, n_features)
        
        # Split recursively
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._build_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feature, best_thresh, left, right)
    
    def _best_split(self, X, y, n_features):
        best_gain = -1
        split_idx, split_thresh = None, None
        
        for feature_idx in range(n_features):
            X_column = X[:, feature_idx]
            thresholds = np.unique(X_column)
            
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)
                
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature_idx
                    split_thresh = threshold
        
        return split_idx, split_thresh
    
    def _information_gain(self, y, X_column, threshold):
        parent_entropy = self._entropy(y)
        
        left_idxs, right_idxs = self._split(X_column, threshold)
        n, n_left, n_right = len(y), len(left_idxs), len(right_idxs)
        
        if n_left == 0 or n_right == 0:
            return 0
        
        child_entropy = (n_left / n) * self._entropy(y[left_idxs]) + (n_right / n) * self._entropy(y[right_idxs])
        
        ig = parent_entropy - child_entropy
        return ig
    
    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs
    
    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])
    
    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]

class RandomForestFromScratch:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2):
        self.n_trees = n_trees
        self.trees = [DecisionTreeFromScratch(max_depth, min_samples_split) for _ in range(n_trees)]
    
    def fit(self, X, y):
        self.X, self.y = X, y
        self.samples = [self._bootstrap_sample(X, y) for _ in range(self.n_trees)]
        
        for i, tree in enumerate(self.trees):
            X_sample, y_sample = self.samples[i]
            tree.fit(X_sample, y_sample)
    
    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        return [Counter(tree_pred).most_common(1)[0][0] for tree_pred in tree_preds]
    
    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]
